# 1. 동별 인구수에 따라 최적의 수거기 수 결정하기

In [3]:
import pandas as pd
import copy

# 광양시 동별인구수 데이터 불러오기
df = pd.read_csv('/content/11.광양시_동별인구수_세대수.csv')

# deep copy로 서로 다른 주소로 저장
remove_gumho_df = copy.deepcopy(df)

# 금호동 제거
geomho = remove_gumho_df.iloc[10]
remove_gumho_df.drop(remove_gumho_df.index[(remove_gumho_df['emd_nm'] == '금호동')], axis=0, inplace=True)
remove_gumho_df.head()

# 동 인구수에 따라 수거기 수 나누기
machine = 50 - 12
dong = len(remove_gumho_df) # 전체 동 수
sum_2017 = sum(remove_gumho_df['total_pop_2017']) # 전체 인구 수

a = []
percent = []
for i in range(dong):
  percent += [(remove_gumho_df.iloc[i][1] * (machine / sum_2017))]
  a += [(round(remove_gumho_df.iloc[i][1] * (machine / sum_2017)))]

  if a[i] > 10:
    a[i] = a[i]-1
a.insert(10, 12)

print(f'===== 동별 수거기 수 리스트 =====\n {a}')

# 원본에 '수거기수' 컬럼 추가하기
df["수거기수"] = a
df["수거기수"]

===== 동별 수거기 수 리스트 =====
 [12, 1, 1, 1, 1, 1, 1, 1, 15, 3, 12, 1]


0     12
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8     15
9      3
10    12
11     1
Name: 수거기수, dtype: int64

# 2. 광양시 동별 공원 수 알아보기

In [5]:
# 동별 공원 정보 데이터 불러오기
df2 = pd.read_csv('/content/전라남도_광양시_도시공원정보_20200928_1601253261583_19062.csv', encoding = 'cp949')

# 동별 공원 수 분포 알아보기
# emd_nm : 동 이름
# 중동, 마동 → 중마동
all = {}
for i in df['emd_nm']:
  all[str(i)] = []

for idx in range(len(df2)):
  if '중동' in df2['소재지도로명주소'][idx] :
    all['중마동'].append(df2['소재지도로명주소'][idx])

  elif '마동' in df2['소재지도로명주소'][idx] :
    all['중마동'].append(df2['소재지도로명주소'][idx])
  
  for emd in df['emd_nm']:
    if emd in df2['소재지도로명주소'][idx] :
      all[emd].append(df2['소재지도로명주소'][idx])

print("===== 동별 공원 수 =====")
park = []
for i in df['emd_nm']:
  print(f'{i} : ', len(all[i]))
  park.append(len(all[i]))

===== 동별 공원 수 =====
광양읍 :  38
봉강면 :  0
옥룡면 :  0
옥곡면 :  3
진상면 :  2
진월면 :  1
다압면 :  0
골약동 :  0
중마동 :  36
광영동 :  10
금호동 :  0
태인동 :  4


# 3. 동별로 공원 총 면적 알아보기

In [6]:
# 동별로 공원 면적 합 구하기
sum_park = {}
for i in df['emd_nm']:
  sum_park[str(i)] = []

for idx in range(len(df2)):
  if '중동' in df2['소재지도로명주소'][idx] :
    sum_park['중마동'].append(df2['공원면적'][idx])

  elif '마동' in df2['소재지도로명주소'][idx] :
    sum_park['중마동'].append(df2['공원면적'][idx])
  
  for emd in df['emd_nm']:
    if emd in df2['소재지도로명주소'][idx] :
      sum_park[emd].append(df2['공원면적'][idx])
park_round = []
for i in df['emd_nm']:
  print(f'{i} : ', round(sum(sum_park[i])))
  park_round.append(round(sum(sum_park[i])))

print("\n공원평 수 :", park_round)

# 원본에 '공원수', '공원평수' 컬럼 추가하기
df['공원수'] = park
df['공원평수'] = park_round

광양읍 :  372966
봉강면 :  0
옥룡면 :  0
옥곡면 :  31836
진상면 :  49485
진월면 :  3000
다압면 :  0
골약동 :  0
중마동 :  841608
광영동 :  36169
금호동 :  0
태인동 :  101783

공원평 수 : [372966, 0, 0, 31836, 49485, 3000, 0, 0, 841608, 36169, 0, 101783]


# 4. 필요한 컬럼만 뽑아서 확인하기

In [7]:
# (인구수, 수거기수, 공원수, 공원총평수)
df_fin = df[['emd_nm', 'total_pop_2017', '수거기수', '공원수', '공원평수']]
df_fin.columns = ['동', '인구수', '수거기수', '공원수', '공원총평수']
df_fin.sort_values('인구수', ascending=False)

,동,인구수,수거기수,공원수,공원총평수
8,중마동,58654,15,36,841608
0,광양읍,50261,12,38,372966
10,금호동,13478,12,0,0
9,광영동,12565,3,10,36169
2,옥룡면,3218,1,0,0
3,옥곡면,3204,1,3,31836
4,진상면,3158,1,2,49485
5,진월면,3132,1,1,3000
1,봉강면,2318,1,0,0
11,태인동,2105,1,4,101783


# 5. 동별 공원의 위도, 경도 알아보기

In [8]:
# 위도
park_sort_lat = {}
for i in df['emd_nm']:
  park_sort_lat[str(i)] = []

for idx in range(len(df2)):
  if '중동' in df2['소재지도로명주소'][idx] :
    park_sort_lat['중마동'].append(df2['위도'][idx])

  elif '마동' in df2['소재지도로명주소'][idx] :
    park_sort_lat['중마동'].append(df2['위도'][idx])
  
  for emd in df['emd_nm']:
    if emd in df2['소재지도로명주소'][idx] :
      park_sort_lat[emd].append(df2['위도'][idx])

# 경도
park_sort_lon = {}
for i in df['emd_nm']:
  park_sort_lon[str(i)] = []

for idx in range(len(df2)):
  if '중동' in df2['소재지도로명주소'][idx] :
    park_sort_lon['중마동'].append(df2['경도'][idx])

  elif '마동' in df2['소재지도로명주소'][idx] :
    park_sort_lon['중마동'].append(df2['경도'][idx])
  
  for emd in df['emd_nm']:
    if emd in df2['소재지도로명주소'][idx] :
      park_sort_lon[emd].append(df2['경도'][idx])

# 위도, 경도
location = {}
for i in df['emd_nm']:
  location[str(i)] = []
  print("=")

  for x,y in zip(park_sort_lat[i], park_sort_lon[i]):
    location[i].append([x,y])
  print(f'{i} : {location[i]}')

=
광양읍 : [[34.985305, 127.58640449999999], [34.96183916, 127.56896870450001], [34.93650533, 127.6050808], [34.9820167, 127.579449], [34.980354, 127.580257], [34.978609000000006, 127.579518], [34.97628182, 127.577155], [34.98062277, 127.577627], [34.97822508, 127.576356], [34.977565999999996, 127.57454399999999], [34.970081, 127.5795669], [34.97160856, 127.580145], [34.973490999999996, 127.5802024], [34.973563, 127.5774187], [34.975072, 127.578946], [34.977988639, 127.581128], [34.978021999999996, 127.58556785629999], [34.9679219604, 127.58281299999999], [34.9692065713, 127.5871528914], [34.9641818396, 127.5729013], [34.9638608904, 127.57552240000001], [34.965804999999996, 127.567649], [34.967240999999994, 127.56323700000002], [34.9661912933, 127.5691870198], [34.976637, 127.60826100000001], [34.97312129, 127.6073218], [34.971245, 127.6076759], [34.970549, 127.611806], [34.969177, 127.6094811], [34.9661912933, 127.5691870198], [34.976760999999996, 127.586496], [34.974555, 127.58888400000